In [423]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv('IRISFeat.csv', header = None)
# print(data.head)
# mins = np.min(data, axis = 0)
# maxs = np.max(data, axis = 0)
# data = 1 - ((maxs - data)/(maxs - mins))
# print(data.head)
index = [i for i in range(data.shape[0])]
np.random.shuffle(index)
data = data.set_index([index]).sort_index()
label = pd.read_csv('IRISlabel.csv', header = None)
label = label.set_index([index]).sort_index()
k = 5
X_shuffled = {}
y_shuffled = {}
for i in range(0,k):
    X_shuffled[i] = data[i::k]
    y_shuffled[i] = label[i::k]

In [424]:
def get_next_train_valid(X_shuffled, y_shuffled, itr):
    X_train = pd.DataFrame()
    y_train = pd.DataFrame()
    X_valid = pd.DataFrame()
    y_valid = pd.DataFrame()
    for i in range(0,k):
        if i != itr:
            X_train = X_train.append(X_shuffled.get(i))
            y_train = pd.concat([y_train, y_shuffled.get(i)])
        else:
            X_valid = X_shuffled.get(i)
            y_valid = y_shuffled.get(i)
    return X_train, y_train, X_valid, y_valid

In [430]:
def train(X_train, y_train):
    theta = np.zeros((2,1))
    alpha = 0.04
    for i in range(0,1000):
        h = sigmoid(np.dot(X_train, theta))
        theta_d = np.dot(X_train.transpose(),  h - y_train )
        theta = theta - alpha*theta_d/y_train.shape[0]
    return theta;

In [431]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [432]:
def predict(X_valid,theta):
    y_predict_class = sigmoid(np.dot(X_valid,theta))
    for i in range(0,y_predict_class.shape[0]):
        if y_predict_class[i] > 0.5:
            y_predict_class[i] = 1
        else:
            y_predict_class[i] = 0
    return y_predict_class

In [433]:
def calcError(real, predicted):
    return np.mean(predicted != real)

In [434]:
train_e = 0
test_e = 0
train_set = []
test_set = []
for i in range(0,k):
    X_train, y_train, X_valid, y_valid = get_next_train_valid(X_shuffled, y_shuffled, i)
    theta = train(X_train,y_train)
    predicted = predict(X_valid,theta)
    predicted_train = predict(X_train,theta)
    train_set.append(calcError(predicted_train,y_train))
    test_set.append(calcError(predicted,y_valid))
    train_e += train_set[-1]
    test_e += test_set[-1]

In [435]:
train_e/k

0    0.006667
dtype: float64

In [436]:
test_e/k

0    0.006667
dtype: float64

In [406]:
train_set

[0    0.008333
 dtype: float64, 0    0.008333
 dtype: float64, 0    0.008333
 dtype: float64, 0    0.008333
 dtype: float64, 0    0.0
 dtype: float64]

In [407]:
test_set

[0    0.0
 dtype: float64, 0    0.0
 dtype: float64, 0    0.0
 dtype: float64, 0    0.0
 dtype: float64, 0    0.033333
 dtype: float64]